<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introducción" data-toc-modified-id="Introducción-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introducción</a></span></li><li><span><a href="#Configuración" data-toc-modified-id="Configuración-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Configuración</a></span></li><li><span><a href="#Definición-de-una-herramienta-en-RAPID" data-toc-modified-id="Definición-de-una-herramienta-en-RAPID-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Definición de una herramienta en RAPID</a></span></li></ul></div>

# Introducción

# Configuración

In [4]:
import numpy as np
import pandas as pd

# MATLAB API
import matlab.engine

In [5]:
import io
from IPython.core.magic import register_cell_magic
ip = get_ipython()

out = io.StringIO()
err = io.StringIO()

# Setup matlab cell magic #
@register_cell_magic
def matlab_magic(line,cell):
    out.truncate(0)
    out.seek(0)
    err.truncate(0)
    err.truncate(0)
    raw = r'{line}.eval("""{cell}""", nargout=0, stdout=out, stderr=err)'
    ip.run_cell(raw.format(line=line, cell=cell))
    print(out.getvalue())
    print(err.getvalue())

In [6]:
eng = matlab.engine.start_matlab() # start matlab session
eng.eval("format compact", nargout=0) # compact formatting in matlab's stdout

# Definición de una herramienta en RAPID

Para definir la herramienta, podemos utilzar el tipo de dato `tooldata`. Como el robot sostiene la herramienta `robhold` será `TRUE`. Para la componente `tframe` definiremos la posicion del TCP (en mm), expresada en el sistema de coordenadas de la muñeca `tool0`, así como su orientación. Por último la componente `loaddata`, que caracteriza los parametros dinámicos de la herramienta, viene definida a su vez por cuatro subcomponentes: `mass` para la masa(en kg), `cog` para el centro de gravedad(en mm), `aom` para la orientación de los ejes de inercia e `ix` `iy` `iz` para los valores de dichas inercias(en $kgm^2$).
<br><br>
Así pues, la estructura desglosada de `tooldata` tiene el siguiente aspecto:

```
< dataobject of tooldata >
    < robhold of bool >
    < tframe of pose >
    < trans of pos >
        < x of num >
        < y of num >
        < z of num >
    < rot of orient >
        < q1 of num >
        < q2 of num >
        < q3 of num >
        < q4 of num >
    < tload of loaddata >
        < mass of num >
        < cog of pos >
            < x of num >
            < y of num >
            < z of num >
        < aom of orient >
            < q1 of num >
            < q2 of num >
            < q3 of num >
            < q4 of num >
        < ix of num >
        < iy of num >
        < iz of num >
```

Se nos pide lo siguiente:
<img style="float: right; padding-right: 80px; padding-top: 40px;" src="./images/ejemplo_puntero.png">
>A la vista de la estructura del tipo de dato tooldata, defínase sobre el
papel una herramienta de tipo puntero, considerando una punta de **105
mm de longitud** y **140 gramos de peso**, con su centro de gravedad
situado a **20 mm de la base** y con unas inercias en su cdg equivalentes
a las que tendría una **varilla del mismo peso y longitud respecto de su
cdg**. En cuanto a la orientación del TCP, se pide definirla a través del
correspondiente cuaternio de **2 formas diferentes**:
- Invirtiendo la orientación de los ejes X y Z del TCP por defecto, pero manteniendo la orientación del eje Y.
- Invirtiendo la orientación del eje Z del TCP por defecto e
intercambiando los ejes X e Y.

<br>
donde se han resaltao algunos parámetros que usaremos para la definición de la herramienta. La varilla, supuesta infinitamente delgada a efectos de cálculos, tiene por lo tanto inercia cero en el eje longitudinal $z$ y, en los ejes transversales $x$ e $y$ viene dada por:
<br>
$$I_x=I_y=\frac{m\ell^{2}}{12}=\frac{0.140\times 0.02^{2}}{12}= 4.66 \times10^{-6} kgm^{2}$$
<br>
donde $m$ y $\ell$ son la masa y la longitud del centro de masa a la base, respectivamente.
<br><br>
RAPID pide al usuario expresar la orientación en forma de quaternios, por lo tanto, distinguiendo entre las dos opciones:<br>

> - Invirtiendo la orientación de los ejes X y Z del TCP por defecto, pero manteniendo la orientación del eje Y.

Que podemos traducir en una rotación de $\frac{\pi}{2}$ sobre el eje $y$. El quaternio viene dado por:
$$ \mathbf{Q} =\left [ \textrm{cos}\frac{\theta}{2}, \mathbf{k}\, \textrm{sin}\frac{\theta}{2} \right ] = \left [ \textrm{cos}\frac{\pi}{4}, \left( 0, 1, 0\right)\, \textrm{sin}\frac{\pi}{4} \right ]= \left [ 0.707,\, 0,\, 0.707,\, 0\right ] $$

> - Invirtiendo la orientación del eje Z del TCP por defecto e intercambiando los ejes X e Y.

Que podriamos expresar como composición de cuaternios. Sin embargo, es cómodo expresar una matriz de rotación y hallar las componentes de $\mathbf{Q}$:
$$ \mathbf{R} = \left [ \mathbf{n\,o\,a}\right ] =
\begin{bmatrix}
0 &1  &0 \\ 
1 &0  &0 \\ 
0 &0  &-1 
\end{bmatrix} $$
y sabiendo que
<br>
$$ q_{0}=\frac{1}{2} \sqrt{n_{x}+o_{y}+a_{z}+1} $$
$$ q_{1}=\textrm{sign}( o_z-a_y )\frac{1}{2} \sqrt{n_{x}-o_{y}-a_{z}+1} $$
$$ q_{2}=\textrm{sign}( a_x-n_z )\frac{1}{2} \sqrt{-n_{x}+o_{y}-a_{z}+1} $$
$$ q_{3}=\textrm{sign}( n_y-o_x )\frac{1}{2} \sqrt{-n_{x}-o_{y}+a_{z}+1} $$
<br>
donde $\textrm{sign}$ toma el signo de su argumento. Sustituyendo obtenemos
<br>
$$
q_{0}=\frac{1}{2} \sqrt{0+0-1+1}= 0\\
q_{1}=\textrm{sign}( 0-0 )\frac{1}{2} \sqrt{0-0+1+1}=0.707 \\
q_{2}=\textrm{sign}( 0-0 )\frac{1}{2} \sqrt{-0+0+1+1}=0.707 \\
q_{3}=\textrm{sign}( 1-1 )\frac{1}{2} \sqrt{-0-0-1+1}=0 \\
$$
<br>
Y por lo tanto,
$$ \mathbf{Q} = \left [ 0,\, 0.707,\, 0.707,\, 0\right ] $$
<br>
Verifiquemos con MATLAB:
<br>


In [9]:
%%matlab_magic eng

Q1 = eul2quat([0 0 pi/2], 'XYZ')

R = [0 1 0; 1 0 0; 0 0 -1];
Q2 = rotm2quat(R)

Q1 =
    0.7071         0         0    0.7071
Q2 =
         0    0.7071    0.7071         0




Como queriamos demostrar.
<br><br>
Ahora podemos definir ambas herramientas en RAPID:
<br>
<p style = "text-indent : 4em;">
    <code> PERS tooldata puntero1 := [ TRUE, [[0, 0, 105], [0.707, 0, 0, 0.707]], [0.14, [0, 0, 20], [0.707, 0, 0, 0.707], 4.66e-6, 4.66e-6, 0]];</code>
</p>
<p style = "text-indent : 4em;">
    <code> PERS tooldata puntero2 := [ TRUE, [[0, 0, 105], [0, 0.707, 0.707, 0]], [0.14, [0, 0, 20], [0, 0.707, 0.707, 0], 4.66e-6, 4.66e-6, 0]];</code>
</p>

In [10]:
#TODO